In [ ]:
#-------- Loading required packages ----------

from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from datetime import datetime as dt_obj
from datetime import timedelta
import drms
import json
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
from pathlib import Path
import os.path
import os
from datetime import datetime
from scipy import stats
import sunpy.map
from sunpy.coordinates import frames
from sunpy.cm import color_tables as ct
from sunpy.timeseries import TimeSeries
from sunpy.time import TimeRange, parse_time
from sunpy.net import hek, Fido, attrs as a
from shutil import copyfile
import tarfile
import time
from tqdm import tqdm, trange
import urllib
import urllib.parse as parse


In [ ]:
# #### ============ REQUEST PART =================
#--------------------------------------------To be changed
datadir = '/MS/ab15ms120/ML/final_updated/codes/'
indir = '/MS/ab15ms120/ML/final_updated/'
ardatafile = 'active_harp_toe_long_lat.txt'        # dataset for download
#--------------------------------------------
col_names  =['harp','toe','lon','lat'] # columns in the input file
ardata     = pd.read_csv(datadir + ardatafile,sep='\t',header=None,names=col_names) # read to dataframe

#--columns of input file---
noaa_list  = ardata['harp']
#time_list  = ardata['start'].str.strip()
date_list = drms.to_datetime(ardata['toe'])


lon_list   = ardata['lon']
lat_list   = ardata['lat']
#----
print(date_list)
count      = len(noaa_list) # number of entries
ardata # printing the dataframe

In [ ]:
# ======== Data export parameters ========

series   = 'hmi.V_45s'  # dataseries
ntime    = 8           # we split the download duration into 56 segments
dur      = '9h'         # each download segment covers 3h data
cad      = '45s'        # cadence for download
cols     = 512          # dimension
rows     = 512
scale    = 0.0301

op       = 'exp_request'
#--------------------------------------------To be changed
email    = 'aditi.hmidatadownload@gmail.com'           # jsoc registered id
#--------------------------------------------
method   = 'url-tar'                  # url or url-tar
protocol = 'FITS,compress Rice'
ffmt     = '{seriesname}.{T_REC:A}.{segment}'
oformat  = 'txt'

JSOC     = 'http://jsoc.stanford.edu'
FETCH    = JSOC + '/cgi-bin/ajax/jsoc_fetch'
# field    = 'field'
# azim     = 'azimuth'
# incl     = 'inclination'
# disamb   = 'disambig'

#--------------------------------------------To be changed
reqfile  = indir + 'requests/' + 'new_active_harp_doppler_req_27_100.txt'   #!!! this file will contain the list of jsoc request id's
errfile  = indir +'requests/' + 'new_active_harp_doppler_errors_27_100.txt'      # the cases of export errors will be saved here
#--------------------------------------------
Path(reqfile).touch()  # create empty request and error files
Path(errfile).touch()
timestep=8;


In [ ]:

#===== Function for requesting jsoc data ========
# inputs:
# i: index string
# noaa : noaa number
# datetime: date, time, and cadence
# lon,lat: longitude and latitude for map projection

def jsoc_request(i, noaa,datetime,lon,lat):
    ds       = '{0}[{1}/{2}@{3}][?Quality>=0?]'.format(series,datetime,dur,cad)

    #--- update the process if required -----
    process  = 'n=0|Maproj,map=Postel,clon={0},clat={1},scale={2},cols={3},rows={4}'    .format(lon,lat,scale,cols,rows)
    #process='n=0|no_op'
    #-----------------------------------------

    RESP     = '/tmp/jsoc_export.'+str(os.getpid()) + '_' + str(i)
    if os.path.exists(RESP):
        os.remove(RESP)

    # url parsing all the inputs
    ds_pq       = parse.quote(ds)
    notify_pq   = parse.quote(email)
    protocol_pq = parse.quote(protocol)
    ffmt_pq     = parse.quote(ffmt)

    # setting up the wget download
    cmd = 'op={0}&ds={1}&process={2}&method={3}&format={4}&protocol={5}&filenamefmt={6}&notify={7}&requestor=none'    .format(op,ds_pq,process,method,oformat,protocol_pq,ffmt_pq,notify_pq)
    #print(cmd)

    toget = '{0} {1}?"{2}"'.format(RESP,FETCH,cmd)
    os.system('wget -S -nv -O {0}'.format(toget))


    #--- reading the request response ---
    f     = open(RESP, 'r')
    line  = f.readline()
    while line:
        print(line,end='')
        line = f.readline()
        if "JSOC_" in line:
            requestid = line.split("=")[1].strip()
        if "size" in line:
            size = line.split("=")[1].strip()
            #print('\nrequestid= ', requestid)
        if "wait" in line:
            wait=line.split("=")[1].strip()

        if "status" in line:
            status=line.split("=")[1].strip()
    f.close()


    # if request fails or there is a queue- wait and note it in the error file
    try:
        in_status = int(status)
    except:
        with open(errfile, 'a') as f:
                f.write('{0}\t{1}\t{2}\t{3}\t{4}\n'.format(i,noaa,datetime,lon,lat))
        return None

    #-- print the request file path
    print('resp= ', RESP)
    print("="*10)
    #----
    # if the run is successful note the request id's in file
    with open(reqfile, 'a') as f:
        try:
            f.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\n'.format(i,noaa,requestid, RESP,size,wait))
        except:
            pass
    return None


In [ ]:

# ====== Loop for the JSOC requests ===========

#--------------------------------------------To be changed
n1 = 0          # noaa start number from the list; set 0 to start from begining
n2 = 100        # noaa end number; set as variable count for taking all the entries
#-------------------------------------------- 
m1 = 0          # start number for download segment; typically 0
m2 = ntime      # end number for download segment; typically ntime 

# the n,m  values can be changed to run for specific cases when error occurs

for i in range(n1, n2):

    dtime_1 = date_list[i]
    noaa  = noaa_list[i]
    lon   = lon_list[i]
    lat   = lat_list[i]
    dtime = dtime_1 + timedelta(hours=1*timestep)
    for j in range(m1, m2):
        print('i, j = {0}, {1}'.format(i, j) )
        dt1 = dtime - timedelta(hours=j*timestep)
        dtf = dt1.strftime('%Y.%m.%d_%H:%M:%S_TAI')
        index = str(i)+'_' + str(j)
        now = datetime.now()
        print('now = ', now)
        jsoc_request(index,noaa,dtf,lon,lat)
        time.sleep(1200)
    #clear_output()
